# Push Docker Image to AWS Elastic Container Registry

In [ ]:
!cd container; ./build_and_push.sh drug-prediction

# Run Local Training Job

In [1]:
# Download the dataset to local

# download pretrained mol2vec model
!wget -O data/model_300dim.pkl https://github.com/samoturk/mol2vec/raw/master/examples/models/model_300dim.pkl model_300dim.pkl   

# download example data     
!wget -O data/ames.sdf https://github.com/samoturk/mol2vec/raw/master/examples/data/ames.sdf
    
#download hiv data
!wget -O data/hiv.zip https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/molnet_publish/hiv.zip
!cd data; unzip hiv.zip

--2020-01-21 19:23:11--  https://github.com/samoturk/mol2vec/raw/master/examples/models/model_300dim.pkl
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/samoturk/mol2vec/master/examples/models/model_300dim.pkl [following]
--2020-01-21 19:23:12--  https://raw.githubusercontent.com/samoturk/mol2vec/master/examples/models/model_300dim.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76786055 (73M) [application/octet-stream]
Saving to: ‘data/model_300dim.pkl’

data/model_300dim.p 100%[===================>]  73.23M  42.8MB/s    in 1.7s    

2020-01-21 19:23:15 (42.8 MB/s) - ‘data/model_300dim.pkl’ saved [76786055/76786055]

--2020-01

In [40]:
from sagemaker import get_execution_role

role = get_execution_role()

In [41]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local


In [42]:
from sagemaker.estimator import Estimator

hyperparameters = {'dev-mode': True}

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='drug-prediction:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file:///home/ec2-user/SageMaker/baseline/data')

Creating tmpi7nqgliu_algo-1-qvagh_1 ... 
Attaching to tmpi7nqgliu_algo-1-qvagh_12mdone
algo-1-qvagh_1  | 2020-01-20 22:35:01,264 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-qvagh_1  | 2020-01-20 22:35:01,265 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-qvagh_1  | Returning the value itself
algo-1-qvagh_1  | 2020-01-20 22:35:01,280 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-qvagh_1  | Returning the value itself
algo-1-qvagh_1  | 2020-01-20 22:35:01,283 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-qvagh_1  | 2020-01-20 22:35:01,296 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-qvagh_1  | Returning the value itself
algo-1-qvagh_1  | 2020-01-20 22:35:01,298 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-qvagh_1  | 2020-01

# Run A Sagemaker Training Job

In [47]:
import sagemaker as sage

sess = sage.Session()

In [48]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'drug-prediction'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

405787280505.dkr.ecr.us-west-2.amazonaws.com/drug-prediction:latest


In [49]:
from sagemaker.estimator import Estimator

data_location = 's3://jdurago-insight-2020a/data'
output_path = 's3://jdurago-insight-2020a/output/baseline/'
max_run_time = 3*60*60 # train for max of 3 hours
hyperparameters = {'dev-mode': False}

# instance_type = 'ml.p2.xlarge'
instance_type = 'ml.m5.4xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                     output_path = output_path,
                     train_max_run=max_run_time)

estimator.fit(data_location)


2020-01-20 23:01:13 Starting - Starting the training job...
2020-01-20 23:01:15 Starting - Launching requested ML instances......
2020-01-20 23:02:17 Starting - Preparing the instances for training...
2020-01-20 23:03:00 Downloading - Downloading input data
2020-01-20 23:03:00 Training - Downloading the training image.....2020-01-20 23:03:46,055 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-20 23:03:46,056 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value False to Json.
Returning the value itself
2020-01-20 23:03:47,517 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value False to Json.
Returning the value itself
2020-01-20 23:03:47,520 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-20 23:03:47,530 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value False to Json.
Returning the value itself
2020-01-20 23:03:47,532 sagemaker-containers INF